In [1]:
import pickle
import gzip
import numpy as np
import scipy
import pandas as pd
import keras
import keras.layers as layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.layers import Input, LSTM, Dense, Lambda, Conv1D, Conv2D, AveragePooling2D, AveragePooling1D, Flatten, MaxPooling2D, MaxPooling1D, Dropout
from keras.models import Model
from keras.models import Sequential
from keras.utils import Sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
from keras.applications import imagenet_utils
from keras import backend as K
from time import time
from keras import losses
from sklearn.metrics import  roc_curve, roc_auc_score, classification_report, confusion_matrix, accuracy_score, mean_absolute_error
import glob
from PIL import Image
import h5py
import random
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA, KernelPCA
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.gaussian_process as gp
import pandas as pd
import pickle
from joblib import dump, load
from sklearn.utils import shuffle
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
import warnings
warnings.filterwarnings('ignore')
!pip install silence_tensorflow

  Created wheel for silence-tensorflow: filename=silence_tensorflow-1.1.1-cp36-none-any.whl size=3743 sha256=7acb228a0da59952e1e1218b276442db486564a9f4cbcea2062cf45c3cce1ba5
  Stored in directory: /root/.cache/pip/wheels/51/0b/35/cf3020764bee61daa81fa249df3a448e3806344a087fc12292
Successfully built silence-tensorflow


In [3]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
import tensorflow as tf

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Jan  7 13:32:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    23W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !rm -R qmc qmc1
    !git clone https://github.com/fagonzalezo/qmc.git
    !mv qmc qmc1
    !mv qmc1/qmc .
else:
    import sys
    sys.path.insert(0, "../")

rm: cannot remove 'qmc': No such file or directory
rm: cannot remove 'qmc1': No such file or directory
Cloning into 'qmc'...
remote: Enumerating objects: 215, done.
remote: Total 215 (delta 0), reused 0 (delta 0), pack-reused 215
Receiving objects: 100% (215/215), 17.06 MiB | 13.81 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [7]:
import qmc.tf.layers as layers
import qmc.tf.models as models

In [ ]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i+1)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/autompg/data/auto.data_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/autompg/data/auto.data_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((200,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X, y = shuffle(X, y, random_state=0)
    MAE_opt = 100
    dim_y = 5

    beta_opt =  21.34380031023014 
    gamma_opt =  0.06838888649948885 
    learning_rate_opt =  0.0006801203171698437 
    alpha_opt =  0.507812442018699 
    dim_x_opt =  376 
    num_eig_opt =  94

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((192,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(7, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=7, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 1
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.2604166666666667
---------------------------------------- 2
50/50 [==============================] - 0s 905us/step
------------MAE = 0.2864583333333333
---------------------------------------- 3
50/50 [==============================] - 0s 979us/step
------------MAE = 0.3020833333333333
---------------------------------------- 4
50/50 [==============================] - 0s 938us/step
------------MAE = 0.2760416666666667
---------------------------------------- 5
50/50 [==============================] - 0s 974us/step
------------MAE = 0.28125
---------------------------------------- 6
50/50 [==============================] - 0s 965us/step
------------MAE = 0.2760416666666667
---------------------------------------- 7
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.2552083333333333
---------------------------------------- 8
50/50 [====================

In [8]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i+1)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/autompg/data/auto.data_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/autompg/data/auto.data_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((200,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X, y = shuffle(X, y, random_state=0)
    MAE_opt = 100
    dim_y = 5

    beta_opt =  20.374432957473363 
    gamma_opt =  0.06079277405130638 
    learning_rate_opt =  0.0006742909129798373 
    alpha_opt =  0.11306539277259398 
    dim_x_opt =  82 
    num_eig_opt =  41

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((192,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(7, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=7, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 1
50/50 [==============================] - 2s 1ms/step
------------MAE = 0.24479166666666666
---------------------------------------- 2
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.2604166666666667
---------------------------------------- 3
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.23958333333333334
---------------------------------------- 4
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.265625
---------------------------------------- 5
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.28125
---------------------------------------- 6
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.2604166666666667
---------------------------------------- 7
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.265625
---------------------------------------- 8
50/50 [==============================] - 0s 1ms/step
--

In [9]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i+1)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/autompg/data/auto.data_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/autompg/data/auto.data_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((200,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X, y = shuffle(X, y, random_state=0)
    MAE_opt = 100
    dim_y = 5

    beta_opt =  20.374432957473363 
    gamma_opt =  0.06079277405130638 
    learning_rate_opt =  0.0006742909129798373 
    alpha_opt =  0.11306539277259398 
    dim_x_opt =  82 
    num_eig_opt =  41

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((192,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(7, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=7, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 1
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.21354166666666666
---------------------------------------- 2
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.2552083333333333
---------------------------------------- 3
50/50 [==============================] - 0s 979us/step
------------MAE = 0.2604166666666667
---------------------------------------- 4
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.2552083333333333
---------------------------------------- 5
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.296875
---------------------------------------- 6
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.23958333333333334
---------------------------------------- 7
50/50 [==============================] - 0s 998us/step
------------MAE = 0.2604166666666667
---------------------------------------- 8
50/50 [=======================

In [10]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i+1)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/autompg/data/auto.data_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/autompg/data/auto.data_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((200,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X, y = shuffle(X, y, random_state=0)
    MAE_opt = 100
    dim_y = 5

    beta_opt =  5.8731898157282 
    gamma_opt =  0.08238408070240301 
    learning_rate_opt =  0.0004115471767149409 
    alpha_opt =  0.20463828596381872 
    dim_x_opt =  968 
    num_eig_opt =  242

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((192,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(7, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=7, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 1
50/50 [==============================] - 0s 2ms/step
------------MAE = 0.2604166666666667
---------------------------------------- 2
50/50 [==============================] - 0s 2ms/step
------------MAE = 0.3177083333333333
---------------------------------------- 3
50/50 [==============================] - 0s 2ms/step
------------MAE = 0.2552083333333333
---------------------------------------- 4
50/50 [==============================] - 0s 2ms/step
------------MAE = 0.2864583333333333
---------------------------------------- 5
50/50 [==============================] - 0s 2ms/step
------------MAE = 0.2604166666666667
---------------------------------------- 6
50/50 [==============================] - 0s 2ms/step
------------MAE = 0.234375
---------------------------------------- 7
50/50 [==============================] - 0s 2ms/step
------------MAE = 0.2552083333333333
---------------------------------------- 8
50/50 [=============================

In [11]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i+1)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/autompg/data/auto.data_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/autompg/data/auto.data_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((200,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X, y = shuffle(X, y, random_state=0)
    MAE_opt = 100
    dim_y = 5

    beta_opt =  15.757888415161853 
    gamma_opt =  0.05679325509005201 
    learning_rate_opt =  0.0007535036193425169 
    alpha_opt =  0.3622970685409973 
    dim_x_opt =  379 
    num_eig_opt =  189

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((192,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(7, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=7, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 1
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.265625
---------------------------------------- 2
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.2552083333333333
---------------------------------------- 3
50/50 [==============================] - 0s 991us/step
------------MAE = 0.24479166666666666
---------------------------------------- 4
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.296875
---------------------------------------- 5
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.3020833333333333
---------------------------------------- 6
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.296875
---------------------------------------- 7
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.2604166666666667
---------------------------------------- 8
50/50 [==============================] - 0s 1ms/step


In [12]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i+1)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/autompg/data/auto.data_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/autompg/data/auto.data_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((200,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X, y = shuffle(X, y, random_state=0)
    MAE_opt = 100
    dim_y = 5

    beta_opt =  17.511083844930226 
    gamma_opt =  0.08409917365718246 
    learning_rate_opt =  0.000953753943839656 
    alpha_opt =  0.1403901923340316 
    dim_x_opt =  87 
    num_eig_opt =  5

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((192,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(7, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=7, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 1
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.22916666666666666
---------------------------------------- 2
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.2604166666666667
---------------------------------------- 3
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.25
---------------------------------------- 4
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.2760416666666667
---------------------------------------- 5
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.23958333333333334
---------------------------------------- 6
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.22395833333333334
---------------------------------------- 7
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.2708333333333333
---------------------------------------- 8
50/50 [==============================

In [8]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i+1)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/autompg/data/auto.data_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/autompg/data/auto.data_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((200,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X, y = shuffle(X, y, random_state=0)
    MAE_opt = 100
    dim_y = 5

    beta_opt =  21.73446352382646 
    gamma_opt =  0.05985745273241999 
    learning_rate_opt =  0.00025571972413244835 
    alpha_opt =  0.12027693464505085 
    dim_x_opt =  595 
    num_eig_opt =  148

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((192,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(7, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=7, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 1
50/50 [==============================] - 2s 1ms/step
------------MAE = 0.2552083333333333
---------------------------------------- 2
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.2708333333333333
---------------------------------------- 3
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.2708333333333333
---------------------------------------- 4
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.2760416666666667
---------------------------------------- 5
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.25
---------------------------------------- 6
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.234375
---------------------------------------- 7
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.28125
---------------------------------------- 8
50/50 [==============================] - 0s 1ms/step
--------

In [9]:
BIG_MAE=[]
for i in range(20):
    print('----------------------------------------',i+1)
    number = str(i+1)
    train_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/autompg/data/auto.data_train_5.'
    test_path = '/content/drive/My Drive/Colab_Notebooks/ICML2021/autompg/data/auto.data_test_5.'
    
    data = pd.read_csv(train_path+number, header = None, sep = ' ')
    X = data.to_numpy()
    y = X[:,-1]
    X = X[:,0:-1]

    y = (y-1)/4
    y = np.float64(y)
    y = y.reshape((200,1))
    X = np.float64(X)

    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X)
    X, y = shuffle(X, y, random_state=0)
    MAE_opt = 100
    dim_y = 5

    beta_opt =  18.759383686229413 
    gamma_opt =  0.04184238445456803 
    learning_rate_opt =  0.0006725883038526733 
    alpha_opt =  0.17437780534646607 
    dim_x_opt =  16 
    num_eig_opt =  4

    X_train = X
    y_train = y

    X_test = pd.read_csv(test_path+number, header = None, sep = ' ')
    X_test = X_test.to_numpy()
    X_test = np.float64(X_test)
    y_test = X_test[:,-1]
    X_test = X_test[:,0:-1]
    X_test = scaler.transform(X_test)
    y_test = (y_test-1)/4
    y_test = np.float64(y_test)
    y_test = y_test.reshape((192,1))

    fm_y = layers.QFeatureMapSmp(dim=dim_y, beta=beta_opt)
    fm_x = layers.QFeatureMapRFF(7, dim=dim_x_opt, gamma=gamma_opt, random_state=1)
    qmr2 = models.QMRegressorSGD(input_dim=7, dim_x=dim_x_opt, num_eig=num_eig_opt, dim_y=dim_y, gamma=gamma_opt, random_state=17)

    # for initialize weights

    qmr = models.QMRegressor(fm_x=fm_x, fm_y=fm_y, dim_x=dim_x_opt, dim_y=dim_y)
    qmr.compile()
    qmr.fit(X_train, y_train, epochs=1, batch_size = 4)

    qmr2.set_rho(qmr.get_rho())
    # Train model with SGD
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)
    qmr2.layers[0].trainable = True

    def loss(y_true, y_pred):
        return tf.keras.losses.mean_squared_error(y_true, y_pred[:,0:1]) + alpha_opt * y_pred[:, 1:2]

    qmr2.compile(optimizer, loss=loss)
    qmr2.set_rho(qmr.get_rho())
    early_stop = EarlyStopping( monitor="val_loss",
                                min_delta=0,
                                patience=5,
                                verbose=0,
                                mode="auto",
                                restore_best_weights=True,
                                )

    qmr2.fit(X_train, y_train, epochs=100, validation_split = 0.2, batch_size = 4, callbacks = early_stop, verbose = 0)

    out = qmr2.predict(X_test, batch_size = 4)
    y_pred, std = out[:, 0], np.sqrt(out[:, 1])
    print('------------MAE =', mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))
    BIG_MAE.append(mean_absolute_error(np.rint(y_test*4)+1, np.rint(y_pred*4)+1))

print(np.mean(BIG_MAE))
print(np.std(BIG_MAE))

---------------------------------------- 1
50/50 [==============================] - 0s 952us/step
------------MAE = 0.2552083333333333
---------------------------------------- 2
50/50 [==============================] - 0s 945us/step
------------MAE = 0.296875
---------------------------------------- 3
50/50 [==============================] - 0s 978us/step
------------MAE = 0.2552083333333333
---------------------------------------- 4
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.23958333333333334
---------------------------------------- 5
50/50 [==============================] - 0s 1ms/step
------------MAE = 0.28125
---------------------------------------- 6
50/50 [==============================] - 0s 965us/step
------------MAE = 0.2760416666666667
---------------------------------------- 7
50/50 [==============================] - 0s 989us/step
------------MAE = 0.2708333333333333
---------------------------------------- 8
50/50 [=============================